In [1]:
import pandas as pd
import emoji
import regex as re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from deep_translator import GoogleTranslator


import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/natalie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natalie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
data = pd.read_csv('scrape/restaurant-data/cleaned_restaurant_reviews.csv', index_col=0)

In [5]:
data

,url,name,neighbourhood,price,categories,review,user,date,cleaned_price,cleaned_categories,...,Accessible,Indian,Japanese,Noodles,Sustainable,Desserts,European,Bites,Alcohol,region
0,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']","\nFish Ball Minced Meat Noodle\nFishball, meat...",Triffany Lim,21m ago,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,0,0,0,East
1,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']","\nOrh lua\nThere are a couple of stores, but g...",Ally Tan,Jul 30 at 4:12pm,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,0,0,0,East
2,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']",\nPeanut sauce was ace\nI love a good satay pe...,Ally Tan,Jul 30 at 4:10pm,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,0,0,0,East
3,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']",\nClassic BBQ wings\nJuicy and tasty like it’s...,Ally Tan,Jul 30 at 4:09pm,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,0,0,0,East
4,https://www.burpple.com/bedok-85-market?bp_ref...,85 Fengshan Centre,Bedok,~$5/pax,"['Hawker Food', 'Supper', 'Cheap & Good']",\nBBQ stingray\nIt was yummy but slight warnin...,Ally Tan,Jul 30 at 4:08pm,5.0,"['Local Delights', 'Supper', 'Value']",...,0,0,0,0,0,0,0,0,0,East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,https://www.burpple.com/chui-huay-lim-teochew-...,Chui Huay Lim Teochew Cuisine,Newton,~$50/pax,"['Chinese', 'Good For Groups']",\nBento Box D $13.80\n川椒雞柳 | 鮮腐竹蝦球 | 清炒西蘭花 | ...,K T,"Oct 27, 2020",50.0,"['Chinese', 'Accessible']",...,1,0,0,0,0,0,0,0,0,Central
15,https://www.burpple.com/chui-huay-lim-teochew-...,Chui Huay Lim Teochew Cuisine,Newton,~$50/pax,"['Chinese', 'Good For Groups']",\nBento Box C $11.80\n普寧豆醬走地雞 | 鮮菌翡翠豆腐 | 蒜茸炒四...,K T,"Oct 26, 2020",50.0,"['Chinese', 'Accessible']",...,1,0,0,0,0,0,0,0,0,Central
16,https://www.burpple.com/chui-huay-lim-teochew-...,Chui Huay Lim Teochew Cuisine,Newton,~$50/pax,"['Chinese', 'Good For Groups']",\nBento Box B $11.80\n蒜子豆豉凉瓜黑豬梅肉 | 香菌扒豆腐 | 蒜茸...,K T,"Oct 9, 2020",50.0,"['Chinese', 'Accessible']",...,1,0,0,0,0,0,0,0,0,Central
17,https://www.burpple.com/chui-huay-lim-teochew-...,Chui Huay Lim Teochew Cuisine,Newton,~$50/pax,"['Chinese', 'Good For Groups']",\n潮州糜 Bento A $12.80\n鹵鴨拼豆干 | 川椒雞 | 欖菜四季苗| 菜脯...,K T,"Oct 1, 2020",50.0,"['Chinese', 'Accessible']",...,1,0,0,0,0,0,0,0,0,Central


## Clean

In [6]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [20]:
def clean_text(filename, deemojize=False, clean_punctuation=False, remove_stopwords=False, lemmatize=False, stemming=False):
    df = pd.read_csv(filename, index_col=0)
    text_list = df['review']

    stopwords = nltk.corpus.stopwords.words('english')
    new_stopwords = ['Address', 'Note', 'Tel', 'Website', 'Open']
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    cleaned_text_list = []
    count = 0
    for text in text_list:

        # lower case
        text = text.lower()
    
        if deemojize:
            text = emoji.demojize(text)
    
        if clean_punctuation:
            text = re.sub(r'[^\w\s]', '', text)

        # translate to english
        text = GoogleTranslator(source='auto', target='en').translate(text)
    
        # tokenize
        tokens = word_tokenize(text)

        if remove_stopwords:
            tokens = [word for word in tokens if word not in stopwords]

        if lemmatize:
            # POS tagging
            tokens = [nltk.pos_tag([word]) for word in tokens]
            # lemmatization
            tokens = [lemmatizer.lemmatize(word[0][0], get_wordnet_pos(word[0][1])) 
                if get_wordnet_pos(word[0][1])!=None else lemmatizer.lemmatize(word[0][0]) for word in tokens]
    
        if stemming:
            tokens = [ps.stem(word) for word in tokens]
    
        # concatenate tokens back
        cleaned_text = " ".join(tokens)
        cleaned_text_list.append(cleaned_text)

        if count%100 == 0:
            print(count)
        count+=1

    df['cleaned_text'] = cleaned_text_list
    
    return df

In [21]:
cleaned_df = clean_text('scrape/restaurant-data/cleaned_restaurant_reviews.csv', deemojize=True, clean_punctuation=True, remove_stopwords=True, lemmatize=True, stemming=False)

0


In [19]:
cleaned_df['cleaned_text'].iloc[-2]

'潮州糜 bento 1280 鹵鴨拼豆干 川椒雞 欖菜四季苗 菜脯煎蛋 潮州糜 braise duck beancurd wok fry chicken fillet szechuan peppercorn stir fry french bean mince pork preserve olive leaf preserve radish omelette teochew porridge'

In [22]:
translator = Translator()
translator.translate('안녕하세요.', dest='en')

AttributeError: 'NoneType' object has no attribute 'group'

## LDA